In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [4]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [5]:
SIZE = 64
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(), 
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE),
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [6]:
# Training
batch_size = 128
train_imagefolder = LabImageFolder('../../datasets/dataset/train/colour')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder('../../datasets/dataset/test/colour' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [13]:
kernel_size=3
stride_en=2
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
class ColorizationNet(nn.Module):
  def __init__(self, input_size=128):
    super(ColorizationNet, self).__init__()
    
    self.encoder = nn.Sequential(       
      nn.Conv2d(1, 16, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.Conv2d(16, 32, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32, 64, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(64),
      nn.ReLU(),   
      nn.Conv2d(64, 64, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(64),
      nn.ReLU(),   
    )     
        
    self.decoder = nn.Sequential(  
      nn.ConvTranspose2d(64, 64, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Upsample(scale_factor=scale_factor),   
      nn.ConvTranspose2d(64, 32, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Upsample(scale_factor=scale_factor),   
      nn.ConvTranspose2d(32, 16, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.Upsample(scale_factor=scale_factor),
      nn.ConvTranspose2d(16, 8, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(8),
      nn.ReLU(),
      nn.ConvTranspose2d(8, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode),
      nn.Upsample(scale_factor=scale_factor)
    )

  def forward(self, input):

    encoder = self.encoder(input)
    # Upsample to get colors
    output = self.decoder(encoder)
    return output

In [14]:
model = ColorizationNet()

In [15]:
criterion = nn.MSELoss()

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [17]:
if use_gpu: 
    criterion = criterion.cuda()
    model = model.cuda()

In [18]:
from torchsummary import summary

summary(model, (1, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             160
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 32, 16, 16]           4,640
       BatchNorm2d-5           [-1, 32, 16, 16]              64
              ReLU-6           [-1, 32, 16, 16]               0
            Conv2d-7             [-1, 64, 8, 8]          18,496
       BatchNorm2d-8             [-1, 64, 8, 8]             128
              ReLU-9             [-1, 64, 8, 8]               0
           Conv2d-10             [-1, 64, 4, 4]          36,928
      BatchNorm2d-11             [-1, 64, 4, 4]             128
             ReLU-12             [-1, 64, 4, 4]               0
  ConvTranspose2d-13             [-1, 64, 4, 4]          36,928
      BatchNorm2d-14             [-1, 6

In [19]:
class AverageMeter(object):
  '''A handy class from the PyTorch ImageNet tutorial''' 
  def __init__(self):
    self.reset()
  def reset(self):
    self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
  '''Show/save rgb image from grayscale and ab channels
     Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
  plt.clf() # clear matplotlib 
  color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
  color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
  color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
  color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
  color_image = lab2rgb(color_image.astype(np.float64))
  grayscale_input = grayscale_input.squeeze().numpy()
  if save_path is not None and save_name is not None: 
    plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
    plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [20]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'

In [21]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = AverageMeter()

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = criterion(output_ab, ab)
        
        _loss.update(loss.item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print('Validate: Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
         loss=_loss))

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("Loss/test", _loss.avg, epoch)
    return _loss.avg

In [22]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = AverageMeter()
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = criterion(output_ab, ab) 
        loss.backward()
        optimizer.step()
        
        _loss.update(loss.item(), gray.size(0))
        
    print(f'Epoch: {epoch}, Loss {_loss.val:.4f} ({_loss.avg:.4f})')
    
    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("Loss/train", _loss.avg, epoch)

In [24]:
# Make folders and set parameters
checkpoints = 'checkpoints'
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = 1e10
epochs = 500

In [25]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses < best_losses:
        best_losses = losses
        torch.save(model.state_dict(), '{}/model-epoch-{}-losses-{:.3f}.pth'.format(checkpoints,epoch+1,losses))

Starting training epoch 0
Epoch: 0, Loss 0.0102 (0.0423)
Finished training epoch 0


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 49 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 29 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 45 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 39 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 72 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 222 pixels
  return func(*args, **kwarg

Validate: Loss 0.0106 (0.0103)	
Finished validation.
Starting training epoch 1
Epoch: 1, Loss 0.0088 (0.0093)
Finished training epoch 1


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 38 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 17 pixels
  return func(*args, **kwargs)


Validate: Loss 0.0088 (0.0086)	
Finished validation.
Starting training epoch 2
Epoch: 2, Loss 0.0104 (0.0082)
Finished training epoch 2
Validate: Loss 0.0072 (0.0074)	
Finished validation.
Starting training epoch 3
Epoch: 3, Loss 0.0071 (0.0076)
Finished training epoch 3
Validate: Loss 0.0069 (0.0070)	
Finished validation.
Starting training epoch 4
Epoch: 4, Loss 0.0067 (0.0073)
Finished training epoch 4
Validate: Loss 0.0068 (0.0068)	
Finished validation.
Starting training epoch 5
Epoch: 5, Loss 0.0069 (0.0072)
Finished training epoch 5
Validate: Loss 0.0068 (0.0068)	
Finished validation.
Starting training epoch 6
Epoch: 6, Loss 0.0081 (0.0071)
Finished training epoch 6
Validate: Loss 0.0066 (0.0067)	
Finished validation.
Starting training epoch 7
Epoch: 7, Loss 0.0072 (0.0070)
Finished training epoch 7
Validate: Loss 0.0065 (0.0068)	
Finished validation.
Starting training epoch 8
Epoch: 8, Loss 0.0070 (0.0070)
Finished training epoch 8
Validate: Loss 0.0068 (0.0067)	
Finished validat

...

/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 92 pixels
  return func(*args, **kwargs)


Validate: Loss 0.0084 (0.0076)	
Finished validation.
Starting training epoch 403
Epoch: 403, Loss 0.0027 (0.0028)
Finished training epoch 403
Validate: Loss 0.0081 (0.0073)	
Finished validation.
Starting training epoch 404
Epoch: 404, Loss 0.0029 (0.0028)
Finished training epoch 404
Validate: Loss 0.0085 (0.0078)	
Finished validation.
Starting training epoch 405
Epoch: 405, Loss 0.0034 (0.0027)
Finished training epoch 405
Validate: Loss 0.0082 (0.0075)	
Finished validation.
Starting training epoch 406
Epoch: 406, Loss 0.0029 (0.0028)
Finished training epoch 406
Validate: Loss 0.0080 (0.0073)	
Finished validation.
Starting training epoch 407
Epoch: 407, Loss 0.0028 (0.0028)
Finished training epoch 407
Validate: Loss 0.0078 (0.0073)	
Finished validation.
Starting training epoch 408
Epoch: 408, Loss 0.0028 (0.0028)
Finished training epoch 408
Validate: Loss 0.0083 (0.0076)	
Finished validation.
Starting training epoch 409
Epoch: 409, Loss 0.0026 (0.0028)
Finished training epoch 409
Valida

Validate: Loss 0.0078 (0.0073)	
Finished validation.
Starting training epoch 461
Epoch: 461, Loss 0.0027 (0.0027)
Finished training epoch 461
Validate: Loss 0.0079 (0.0074)	
Finished validation.
Starting training epoch 462
Epoch: 462, Loss 0.0028 (0.0027)
Finished training epoch 462
Validate: Loss 0.0085 (0.0079)	
Finished validation.
Starting training epoch 463
Epoch: 463, Loss 0.0028 (0.0027)
Finished training epoch 463
Validate: Loss 0.0080 (0.0075)	
Finished validation.
Starting training epoch 464
Epoch: 464, Loss 0.0027 (0.0027)
Finished training epoch 464
Validate: Loss 0.0083 (0.0077)	
Finished validation.
Starting training epoch 465
Epoch: 465, Loss 0.0026 (0.0027)
Finished training epoch 465
Validate: Loss 0.0078 (0.0073)	
Finished validation.
Starting training epoch 466
Epoch: 466, Loss 0.0028 (0.0027)
Finished training epoch 466
Validate: Loss 0.0080 (0.0074)	
Finished validation.
Starting training epoch 467
Epoch: 467, Loss 0.0028 (0.0027)
Finished training epoch 467
Valida

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [ ]:
torch.save(model.state_dict(), '{}/model-epoch-last-losses-{:.3f}.pth'.format(checkpoints,"last",losses))

In [ ]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

In [ ]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()